In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib
import csv
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm_notebook # 오래 걸리는 작업 진행확인용
import requests
import time

In [18]:
api_key = "RGAPI-beefeee6-bcef-480e-bcdc-12d3a372ece0"

api_challenger = "https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
api_grandmaster = "https://na1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
api_master = "https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + api_key
r = requests.get(api_challenger)
league = pd.DataFrame(r.json())
r = requests.get(api_grandmaster)
league = pd.concat([league, pd.DataFrame(r.json())], axis=0)
r = requests.get(api_master)
league = pd.concat([league, pd.DataFrame(r.json())], axis=0)

league.reset_index(inplace=True)
league_entries = pd.DataFrame(dict(league["entries"])).T
league = pd.concat([league, league_entries], axis=1)

league = league.drop(["index", "queue", "name", "leagueId", "entries", "rank"], axis=1) #필요없는 칼럼 드랍

In [12]:
league

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,account_id
0,CHALLENGER,VLXBb6vlZ-dRttDFI-SGtFdxSBMw3NBmoVloptE6ept7nNc,hoidro,635,200,173,False,False,False,False,NaN
1,CHALLENGER,DRE2lNOWqX1RR0CE4BPFummaD0gsAxGwP2C5f3c_jj6-0pc,Gweiss,742,279,260,True,False,False,False,NaN
2,CHALLENGER,AALtFgLTn709QG_0KMCQeRx2hLfcB870ootBc_3Ubq00PNA,ur babe,704,168,130,False,False,False,False,NaN
3,CHALLENGER,7b2IVLXJP8l-3jiANxNnsh26IAvgfGI2pboPp05kL5Q99Uc,Søxo,811,284,247,True,False,False,False,NaN
4,CHALLENGER,z9Hwv_mrGQZiBlSj3RhxKkkGC5ekrGyJ8tkToltjW8i6Qcs,CyanInkstone,680,292,268,False,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2964,MASTER,Q8mGU4on7ZQVjdQzGkpfjklbL8hPTuMwIKMwIEI4HSZh1qI,Kaleido,98,71,57,False,False,False,False,NaN
2965,MASTER,AyxYESw25Xw_7udtio7_EZbKjxDAOMFyXwVH64wbg2CZTW...,April Abril,135,75,48,False,False,False,False,NaN
2966,MASTER,F2YxSeT32waHVVHbYkJkXvMhA1v5UbR_CUuajUyNhZdaW18,Pocket Rhino,211,45,28,False,False,False,False,NaN
2967,MASTER,_jm982lB5melNKPnATfedh2wnufEDQtoObQG_pYj0E2Cmno,Not2Clean,67,412,410,False,False,False,False,NaN


In [22]:
league["account_id"] = np.nan # account_id 초기화
# for i, summoner_id in enumerate(league["summonerId"]):
#     Summoner API에서 AccountId를 가져와 채워넣는다.
api_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/" + 'VLXBb6vlZ-dRttDFI-SGtFdxSBMw3NBmoVloptE6ept7nNc' + "?api_key=" + api_key
r = requests.get(api_url)
# while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
#     time.sleep(5)
#     r = requests.get(api_url)
account_id = r.json()["accountId"]
league.iloc[i, -1] = account_id
league

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,account_id
0,CHALLENGER,WULTnOassx_JZIPX014AH6ipqddZwvo8UKt-OnchZ1LYhgTT,Clam,862,232,198,True,False,False,True,SOCLLcKAoVtbJ4ZDBg59pfNy3W65IhT_EuaygmMvDbGrI2Q
1,CHALLENGER,73icM9fv0AAilNv_0AKO8UeLm-o-EWEvbM2hDECnAkfcJc4,nczxvqewa,610,93,58,False,False,True,False,NaN
2,CHALLENGER,36w2FCICdypZqxBZ9kAyZQ5w1uab_RcG1Dr9CH2lMGX2nSA,C9 eXyu,779,346,327,True,False,False,False,NaN
3,CHALLENGER,EQbsEzxnO0XGphUqP1sGTTRD_eEavy992XUj1OIAVhdJEF...,GG Swip3rR,619,314,285,False,False,True,False,NaN
4,CHALLENGER,7p1j8qUm3SHgnzw1SN6PtGQ-nX96EqY8m-TDQY2CvjaQIW...,Níles,710,239,213,False,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2964,MASTER,QjugMqWoc-HnWZfbxoPDPyBTgyKZKvl2QicIt8Zuo2Mr6eo,guccì,184,69,75,False,False,False,False,NaN
2965,MASTER,2ty8GdxpJANm7JYIuvDTQOMQVid1rYo3n1Dyh4VCKfgSGms,Numb Nightmare,0,833,823,False,False,False,False,NaN
2966,MASTER,bbKf0XiIqyLhiLA8SmrCfV8PT2jGFP5b_ck9I5GPcnlsiiM,Mr Zlatan,135,137,135,False,False,False,False,NaN
2967,MASTER,milcBYlCjzRUVRpKSgDHFMsywIlU2DafaOfSrYMDki2WeQo,weakside wretch,0,74,59,False,False,False,False,NaN


In [23]:
account_id = 'SOCLLcKAoVtbJ4ZDBg59pfNy3W65IhT_EuaygmMvDbGrI2Q'

season = str(13) #13시즌에 치뤄진 경기를 불러온다.

match_info_df = pd.DataFrame()
# for account_id in league_df["account_id"]:
api_url = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + account_id + \
              "?season=" + season + "&api_key=" + api_key
r = requests.get(api_url)
# while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
#     time.sleep(5)
#     r = requests.get(api_url)
match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()["matches"])])
match_info_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,NA1,3907063791,875,400,13,1621051683895,DUO_SUPPORT,BOTTOM
1,NA1,3907059952,777,400,13,1621050419255,DUO_SUPPORT,NONE
2,NA1,3907052103,412,400,13,1621048100601,DUO_SUPPORT,MID
3,NA1,3906948264,79,450,13,1621046589216,DUO_SUPPORT,NONE
4,NA1,3906970546,104,400,13,1621044241596,SOLO,MID
...,...,...,...,...,...,...,...,...
95,NA1,3891673441,85,420,13,1620025174824,SOLO,MID
96,NA1,3891568200,85,420,13,1620022761606,DUO,TOP
97,NA1,3891551896,85,420,13,1620020180458,DUO,TOP
98,NA1,3891318098,143,450,13,1620009732268,DUO_SUPPORT,NONE


In [26]:
match_info_df = match_info_df.drop_duplicates("gameId") #중복 경기기록은 삭제

match_df = pd.DataFrame()
for game_id in match_info_df["gameId"]:
    api_url = "https://na1.api.riotgames.com/lol/match/v4/matches/" + str(game_id) + "?api_key=" + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 오류를 리턴할 경우 지연하고 다시 시도
        time.sleep(30)
        r = requests.get(api_url)
        print('error',game_id)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T
    match_df = pd.concat([match_df, temp_df])

error 3907063791


In [28]:
match_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,3907063791,NA1,1621051683895,1910,400,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3907059952,NA1,1621050419255,943,400,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3907052103,NA1,1621048100601,1803,400,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3906948264,NA1,1621046589216,852,450,12,13,11.10.374.9538,ARAM,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3906970546,NA1,1621044241596,2063,400,11,13,11.10.374.9538,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3891673441,NA1,1620025174824,1892,420,11,13,11.9.372.2066,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3891568200,NA1,1620022761606,1666,420,11,13,11.9.372.2066,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3891551896,NA1,1620020180458,1864,420,11,13,11.9.372.2066,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
0,3891318098,NA1,1620009732268,923,450,12,13,11.9.372.2066,ARAM,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
